### Bring up a cluster
Find the commands presented here ready-to-go in script ```clusterup.sh```

In [60]:
!gcloud container clusters create microblog-cluster --num-nodes=2

This will enable the autorepair feature for nodes. Please see https://cloud.google.com/kubernetes-engine/docs/node-auto-repair for more information on node autorepairs.
Creating cluster microblog-cluster in us-central1-a...done.
Created [https://container.googleapis.com/v1/projects/going-tfx/zones/us-central1-a/clusters/microblog-cluster].
To inspect the contents of your cluster, go to: https://console.cloud.google.com/kubernetes/workload_/gcloud/us-central1-a/microblog-cluster?project=going-tfx
kubeconfig entry generated for microblog-cluster.
NAME               LOCATION       MASTER_VERSION  MASTER_IP      MACHINE_TYPE   NODE_VERSION  NUM_NODES  STATUS
microblog-cluster  us-central1-a  1.10.9-gke.5    35.239.217.33  n1-standard-1  1.10.9-gke.5  2          RUNNING


In [61]:
!gcloud container clusters get-credentials microblog-cluster --zone us-central1-a --project going-tfx

Fetching cluster endpoint and auth data.
kubeconfig entry generated for microblog-cluster.


---
### Prepare DNS records
You still need a domain name service. These records here are for internal routing once the HTTP requests arrive in the google infrastructure

In [43]:
!gcloud compute addresses list

NAME             ADDRESS/RANGE   TYPE  PURPOSE  NETWORK  REGION       SUBNET  STATUS
hello-server-ip  35.186.195.159                                               RESERVED
jenkins          35.244.196.48                                                RESERVED
microblog        35.244.236.101                                               IN_USE
jenkins          35.184.14.235                           us-central1          IN_USE


In [21]:
!gcloud dns record-sets list --zone=fnf-ninja-on-going-tfx

NAME                  TYPE  TTL    DATA
fnf.ninja.            NS    21600  ns-cloud-c1.googledomains.com.,ns-cloud-c2.googledomains.com.,ns-cloud-c3.googledomains.com.,ns-cloud-c4.googledomains.com.
fnf.ninja.            SOA   21600  ns-cloud-c1.googledomains.com. cloud-dns-hostmaster.google.com. 1 21600 3600 259200 300
microblog.fnf.ninja.  A     300    35.244.236.101


In [55]:
!gcloud dns record-sets transaction start --zone=fnf-ninja-on-going-tfx

Transaction started [transaction.yaml].


In [56]:
!gcloud dns record-sets transaction add --zone=fnf-ninja-on-going-tfx --name="microblog.fnf.ninja"\
    --type=A --ttl=300 "35.244.236.101"

Record addition appended to transaction at [transaction.yaml].


In [57]:
!gcloud dns record-sets transaction execute --zone=fnf-ninja-on-going-tfx

ERROR: (gcloud.dns.record-sets.transaction.execute) HTTPError 409: The resource 'entity.change.additions[1]' named 'microblog.fnf.ninja. (A)' already exists


---
### Deploy and serve the application

In [2]:
!kubectl run microblog --image=us.gcr.io/going-tfx/microblog:v1 --port=5000

kubectl run --generator=deployment/apps.v1 is DEPRECATED and will be removed in a future version. Use kubectl run --generator=run-pod/v1 or kubectl create instead.
deployment.apps/microblog created


In [3]:
!kubectl apply -f k8s/service.yaml

service/microblog created


---
This is the ingress for TLS endpoint and the ACME http01 challenge (proof of domain)


In [122]:
!kubectl apply -f k8s/ingress-tls.yaml

ingress.extensions/microblog-ingress-tls unchanged


In [124]:
!kubectl describe -f k8s/ingress-tls.yaml

Name:             microblog-ingress-tls
Namespace:        default
Address:          35.244.236.101
Default backend:  default-http-backend:80 (10.16.0.8:8080)
TLS:
  microblog-fnf-ninja-tls terminates microblog.fnf.ninja
Rules:
  Host  Path  Backends
  ----  ----  --------
  *     
        /   microblog:5000 (<none>)
Annotations:
  ingress.kubernetes.io/target-proxy:                k8s-tp-default-microblog-ingress-tls--aa4c050bb17c0a5c
  ingress.kubernetes.io/url-map:                     k8s-um-default-microblog-ingress-tls--aa4c050bb17c0a5c
  kubectl.kubernetes.io/last-applied-configuration:  {"apiVersion":"extensions/v1beta1","kind":"Ingress","metadata":{"annotations":{"certmanager.k8s.io/cluster-issuer":"letsencrypt-staging","kubernetes.io/ingress.global-static-ip-name":"microblog"},"labels":{"app":"microblog"},"name":"microblog-ingress-tls","namespace":"default"},"spec":{"rules":[{"http":{"paths":[{"backend":{"serviceName":"microblog","servicePort":5000},"path":"/"}]}}],"tls":[{"hos

In [65]:
!kubectl describe service microblog

Name:                     microblog
Namespace:                default
Labels:                   run=microblog
Annotations:              kubectl.kubernetes.io/last-applied-configuration:
                            {"apiVersion":"v1","kind":"Service","metadata":{"annotations":{"service.alpha.kubernetes.io/app-protocols":"{\"my-https-port\":\"HTTPS\"}"...
                          service.alpha.kubernetes.io/app-protocols: {"my-https-port":"HTTPS"}
Selector:                 run=microblog
Type:                     NodePort
IP:                       10.19.246.11
Port:                     <unset>  5000/TCP
TargetPort:               5000/TCP
NodePort:                 <unset>  31709/TCP
Endpoints:                10.16.0.8:5000
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>


---
#### Check if the service can be located through the ingress
Give it a couple of minutes, or check on the Kubernetes console for completion

In [71]:
!curl http://microblog.fnf.ninja | grep Wolfie

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1621  100  1621    0     0  95849      0 --:--:-- --:--:-- --:--:--   98k
<h1>Hello, Wolfie!</h1>


#### Remove it, it will be replaced by a proper TLS ingress

In [72]:
!kubectl delete -f k8s/ingress.yaml

ingress.extensions "microblog-ingress" deleted


---
### Add certificate infrastructure via helm

In [73]:
!kubectl create serviceaccount -n kube-system tiller

serviceaccount/tiller created


In [74]:
!kubectl create clusterrolebinding tiller-binding --clusterrole=cluster-admin --serviceaccount kube-system:tiller

clusterrolebinding.rbac.authorization.k8s.io/tiller-binding created


In [75]:
!helm init --service-account tiller

$HELM_HOME has been configured at /home/wgiersche/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [76]:
!helm repo update

Hang tight while we grab the latest from your chart repositories...
...Skip local chart repository
...Successfully got an update from the "stable" chart repository
Update Complete. ⎈ Happy Helming!⎈ 


---
### Install cert manager

In [94]:
!helm del --purge cert-manager

release "cert-manager" deleted


In [ ]:
#!helm install --name cert-manager --version v0.3.2 --namespace kube-system stable/cert-manager

In [95]:
!helm install --name cert-manager --version v0.5.0 --namespace kube-system stable/cert-manager

Error: release cert-manager failed: customresourcedefinitions.apiextensions.k8s.io "certificates.certmanager.k8s.io" already exists


In [91]:
!kubectl describe deployment -n kube-system cert-manager-cert-manager

Name:                   cert-manager-cert-manager
Namespace:              kube-system
CreationTimestamp:      Sun, 16 Dec 2018 15:05:18 +0000
Labels:                 app=cert-manager
                        chart=cert-manager-v0.3.2
                        heritage=Tiller
                        release=cert-manager
Annotations:            deployment.kubernetes.io/revision: 1
Selector:               app=cert-manager,release=cert-manager
Replicas:               1 desired | 1 updated | 1 total | 0 available | 1 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  25% max unavailable, 25% max surge
Pod Template:
  Labels:           app=cert-manager
                    release=cert-manager
  Service Account:  cert-manager-cert-manager
  Containers:
   cert-manager:
    Image:      quay.io/jetstack/cert-manager-controller:v0.3.0
    Port:       <none>
    Host Port:  <none>
    Args:
      --cluster-resource-namespace=$(POD_NAMESPACE)
      --l

In [79]:
!kubectl apply -f k8s/clusterissuer.yaml

clusterissuer.certmanager.k8s.io/letsencrypt-staging created
clusterissuer.certmanager.k8s.io/letsencrypt-prod created


In [80]:
!kubectl describe -f k8s/clusterissuer.yaml | grep Name:

Name:         letsencrypt-staging
  Cluster Name:        
      Name:  letsencrypt-staging
Name:         letsencrypt-prod
  Cluster Name:        
      Name:  letsencrypt-prod


In [76]:
!kubectl apply -f k8s/certificate.yaml

certificate.certmanager.k8s.io/microblog-fnf-ninja-tls created


In [157]:
!kubectl get pods -n kube-system | grep cert-manager
pod = !kubectl get pods -n kube-system | grep cert-manager
pod = pod[0].split(' ')[0]

cert-manager-cert-manager-698758c6d-fzg2g                     1/1     Running   0          4h


In [159]:
!kubectl logs -n kube-system $pod | tail -120

I1216 21:38:14.644258       1 acme.go:159] getting private key (letsencrypt-staging->tls.key) for acme issuer kube-system/letsencrypt-staging
I1216 21:38:14.644682       1 logger.go:27] Calling GetOrder
I1216 21:38:14.816775       1 logger.go:52] Calling GetAuthorization
I1216 21:38:14.880763       1 logger.go:72] Calling HTTP01ChallengeResponse
I1216 21:38:14.880834       1 prepare.go:263] Cleaning up old/expired challenges for Certificate default/microblog-fnf-ninja-tls
I1216 21:38:14.880878       1 logger.go:47] Calling GetChallenge
I1216 21:38:14.974532       1 http.go:122] wrong status code '404'
I1216 21:38:14.974937       1 ingress.go:33] Looking up Ingresses for selector certmanager.k8s.io/acme-http-domain=1261569893,certmanager.k8s.io/acme-http-token=1014500891
I1216 21:38:14.975093       1 helpers.go:162] Found status change for Certificate "microblog-fnf-ninja-tls" condition "Ready": "False" -> "False"; setting lastTransitionTime to 2018-12-16 21:38:14.975086024 +0000 UTC m=

In [167]:
!kubectl apply -f k8s/certificate.yaml

certificate.certmanager.k8s.io/microblog-fnf-ninja-tls created


In [154]:
!kubectl describe -f k8s/certificate.yaml

Name:         microblog-fnf-ninja-tls
Namespace:    default
Labels:       <none>
Annotations:  <none>
API Version:  certmanager.k8s.io/v1alpha1
Kind:         Certificate
Metadata:
  Cluster Name:        
  Creation Timestamp:  2018-12-16T17:51:04Z
  Generation:          1
  Owner References:
    API Version:           extensions/v1beta1
    Block Owner Deletion:  true
    Controller:            true
    Kind:                  Ingress
    Name:                  microblog-ingress-tls
    UID:                   285d9449-015b-11e9-8336-42010a80016c
  Resource Version:        10337
  Self Link:               /apis/certmanager.k8s.io/v1alpha1/namespaces/default/certificates/microblog-fnf-ninja-tls
  UID:                     285f8931-015b-11e9-8336-42010a80016c
Spec:
  Acme:
    Config:
      Domains:
        microblog.fnf.ninja
      Http 01:
        Ingress:  
  Common Name:    
  Dns Names:
    microblog.fnf.ninja
  Issuer Ref:
    Kind:       ClusterIssuer
    Name:       letsencrypt-stag

---
### Deploy the secure ingress

In [168]:
!kubectl delete -f k8s/ingress-http01.yaml

ingress.extensions "microblog-ingress-http01" deleted


In [169]:
!kubectl apply -f k8s/ingress-tls.yaml

ingress.extensions/microblog-ingress-tls created


In [166]:
!kubectl describe -f k8s/ingress-http01.yaml

Name:             microblog-ingress-http01
Namespace:        default
Address:          35.244.236.101
Default backend:  default-http-backend:80 (10.16.0.7:8080)
Rules:
  Host  Path  Backends
  ----  ----  --------
  *     
        /   microblog:5000 (<none>)
Annotations:
  ingress.kubernetes.io/backends:                    {"k8s-be-31742--1d282262d54dfc76":"HEALTHY","k8s-be-31811--1d282262d54dfc76":"HEALTHY"}
  ingress.kubernetes.io/forwarding-rule:             k8s-fw-default-microblog-ingress-http01--1d282262d54dfc76
  ingress.kubernetes.io/target-proxy:                k8s-tp-default-microblog-ingress-http01--1d282262d54dfc76
  ingress.kubernetes.io/url-map:                     k8s-um-default-microblog-ingress-http01--1d282262d54dfc76
  kubectl.kubernetes.io/last-applied-configuration:  {"apiVersion":"extensions/v1beta1","kind":"Ingress","metadata":{"annotations":{"kubernetes.io/ingress.global-static-ip-name":"microblog"},"labels":{"app":"microblog"},"name":"microblog-ingress-http01",

In [64]:
!kubectl apply -f k8s/ingress-tls.yaml

ingress.extensions/microblog-ingress-tls created


In [162]:
!kubectl get secrets

NAME                  TYPE                                  DATA   AGE
default-token-2f545   kubernetes.io/service-account-token   3      4h


In [58]:
!curl https://microblog.fnf.ninja | grep Wolfie

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (35) Unknown SSL protocol error in connection to microblog.fnf.ninja:443 
